In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from scipy import stats

import probability_distributions
import maximum_nudges
import evolutionary_algorithms as ea
import maximum_nudges_evolutionary as ev_max_nudges
import nudge_non_causal

In [ ]:
#distribution parameters
input_variables = 2
number_of_states = 5
nudge_size = 0.01

#generate both input and conditional output with Dirichlet weights
distribution_shape = [number_of_states]*input_variables
total_number_of_states = reduce(lambda x,y: x*y, distribution_shape)
input_dist = np.random.dirichlet([1]*total_number_of_states)
input_dist = np.reshape(input_dist, distribution_shape)
cond_shape = [number_of_states]*(input_variables+1)
cond_output = [
    probability_distributions.compute_joint_uniform_random((number_of_states,))
    for i in range(number_of_states**(input_variables))
]
cond_output = np.array(cond_output)
cond_output = np.reshape(cond_output, cond_shape)

In [ ]:
#local nudge optimization
number_of_generations = 500 
population_size = 20
number_of_children = 10 
generational = False
mutation_size = nudge_size/4
#parent_selection_mode = "rank_exponential"
parent_selection_mode = None
mutation_size_weights = 0.025
change_mutation_size_weights = 0.2
start_mutation_size = 0.05
change_mutation_size = 0.2
nudged_vars_to_states = {
    nudged_var:number_of_states for nudged_var in range(input_variables)
}
print(nudged_vars_to_states)

In [ ]:

#create the initial population
local_nudges = []
for _ in range(population_size):
    new_local_nudge = ev_max_nudges.LocalNudge.create_local_nudge(
        nudged_vars_to_states, nudge_size, mutation_size_weights,
        start_mutation_size, change_mutation_size, change_mutation_size_weights, 
        timestamp=0
    )
    local_nudges.append(new_local_nudge)

for local_nudge in local_nudges:
    local_nudge.evaluate(input_dist, cond_output)
print("initial impact local nudge {}".format(
    ea.sort_individuals(local_nudges)[0].score
))

#start the optimization process
find_max_local_nudge = ev_max_nudges.FindMaximumLocalNudge(
    input_dist, cond_output, nudge_size, 
    generational, number_of_children, parent_selection_mode
)
max_local_nudge_individual = find_max_local_nudge.get_max_nudge(
    local_nudges, number_of_generations
)
print("the found max impact for a local nudge {}".format(
    max_local_nudge_individual.score
))


### Define evolutionary parameters local nudge 

In [ ]:
local_evolutionary_params = {
    "number_of_generations": 250,
    "population_size": 20,
    "number_of_children": 40, 
    "generational": True,
    "parent_selection_mode": "rank_exponential",
    "mutation_size_weights": 0.2,
    "change_mutation_size_weights": 0.3,
    "start_mutation_size": 0.025,
    "change_mutation_size": 0.3
}

In [ ]:
max_local_nudge = ev_max_nudges.find_maximum_local_nudge(
    input_dist, cond_output, nudge_size, 
    local_evolutionary_params, verbose=True
)
print(max_local_nudge.score)
print(max_local_nudge.weights)


In [ ]:
individual_nudges = max_local_nudge.individual_nudges

nudge_vectors = [
    weight*nudge.genes 
    for nudge, weight in zip(individual_nudges, max_local_nudge.weights)
]
new_dist = nudge_non_causal.nudge_local(
    input_dist, [0, 1], 0.01, nudge_vectors 
)
print(np.sum(np.absolute(input_dist-new_dist)))
print(nudge_non_causal.find_nudge_impact(
    input_dist, new_dist, cond_output,
    measure="absolute"
))

In [ ]:
individual_nudges[0].genes
individual_nudges[1].genes